Google AI Studio

Agente de RH - responder dúvidas dos funcionários

instruir o agente (llm) a escolher algum caminho (triagem)
- Busca informação usando RAG (Retrieval-Augmented Generation)
- Abre ticket
- Pede mais informações


In [2]:
!pip install -q --upgrade langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.2 MB/s eta 0:00:00


In [3]:
# importando API key do google
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GeminiAPI')

In [4]:
# Conexão com o Gemini: ver informações sobre cada modelo no Google AI Studio
llm = ChatGoogleGenerativeAI(
  model ="gemini-2.5-flash",
  temperature=0.0,
  api_key=GOOGLE_API_KEY
)

In [5]:
#
resposta_teste = llm.invoke("Quem é voce?")
print(resposta_teste.content)

Eu sou um modelo de linguagem grande, treinado pelo Google.


In [6]:
# informacao do prompt

triagem_prompt = (
  "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
  "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
  "{\n"
  '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
  '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
  '  "campos_faltantes": ["..."]\n'
  "}\n"
  "Regras:\n"
  '- AUTO_RESOLVER: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
  '- PEDIR_INFO: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
  '- ABRIR_CHAMADO: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
  "Analise a mensagem e decida a ação mais apropriada."
)

In [7]:
# como o prompt vai funcionar

from pydantic import BaseModel, Field #
from typing import Literal, List, Dict #  importa dicionarios explicando as extruturas de dados

# classe com variaveis importantes
class TriagemOUT(BaseModel):
  decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)

In [8]:
llm_triagem = ChatGoogleGenerativeAI(
  model ="gemini-2.5-flash",
  temperature=0.0,
  api_key=GOOGLE_API_KEY
)

In [9]:
from langchain_core.messages import SystemMessage, HumanMessage #diferenciar mensagem do sistema e do usuario

# criando fluxo de triagem
triagem_chain = llm_triagem.with_structured_output(TriagemOUT)

def triagem(mensagem: str) -> Dict:
  saida: TriagemOUT = triagem_chain.invoke([
      SystemMessage(content=triagem_prompt),
      HumanMessage(content=mensagem)
  ])
  return saida.model_dump()


In [10]:
testes = ["posso reembolsar a internet?",
          "quero mais 5 dias de trabalho remoto. como faço?",
          "posso reembolsar cursos ou treinamentos da Alura?",
          "quantas capivaras tem no rio pinheiros?"
          ]

In [11]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: posso reembolsar a internet?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: quero mais 5 dias de trabalho remoto. como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: posso reembolsar cursos ou treinamentos da Alura?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: quantas capivaras tem no rio pinheiros?
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['assunto_da_solicitacao']}



Aula 2

In [12]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [13]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []
for n in Path("/content/drive/MyDrive/estudos/ImersaoDEV").glob("*.pdf"): # filtra só pdf
  try:
    loader = PyMuPDFLoader(str(n)) # guarda pdf e converte em string
    docs.extend(loader.load()) # adiciona o conteudo do arquivo na lista docs
    print(f"Carregado com sucesso arquivo {n.name}") # n.name= nome do arquivo

  except Exception as e: # define o erro como "e"
    print(f"Erro ao carregar arquivo {n.name}: {e}")

print(f"Total de documentos carregados: {len(docs)}")

Carregado com sucesso arquivo Política de Reembolsos (Viagens e Despesas).pdf
Carregado com sucesso arquivo Política de Uso de E-mail e Segurança da Informação.pdf
Carregado com sucesso arquivo Políticas de Home Office.pdf
Total de documentos carregados: 3


In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# transformando bloco de texto em pedaços ("chunks")
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30) # tamanho, contexto

chunks = splitter.split_documents(docs)

In [15]:
# visualizar chunks
for chunk in chunks:
    print(chunk)
    print("------------------------------------")

page_content='Política de Reembolsos (Viagens e 
Despesas) 
 
1.​ Reembolso: requer nota fiscal e deve ser submetido em até 10 dias corridos após a 
despesa.​
 
2.​ Alimentação em viagem: limite de R$ 70/dia por pessoa. Bebidas alcoólicas não 
são reembolsáveis.​' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/content/drive/MyDrive/estudos/ImersaoDEV/Política de Reembolsos (Viagens e Despesas).pdf', 'file_path': '/content/drive/MyDrive/estudos/ImersaoDEV/Política de Reembolsos (Viagens e Despesas).pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}
------------------------------------
page_content='são reembolsáveis.​
 
3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. 
Comprovantes obrigatórios.​
 
4.​ Internet para home

In [16]:
# embedding

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model = "model/gemini/embedding-001",
    google_api_key = GOOGLE_API_KEY
)

In [17]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

# retornar por similaridade
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold":0.3, "k": 4})

GoogleGenerativeAIError: Error embedding content: 400 * BatchEmbedContentsRequest.model: unexpected model name format
* BatchEmbedContentsRequest.requests[0].model: unexpected model name format
* BatchEmbedContentsRequest.requests[1].model: unexpected model name format
* BatchEmbedContentsRequest.requests[2].model: unexpected model name format
* BatchEmbedContentsRequest.requests[3].model: unexpected model name format
* BatchEmbedContentsRequest.requests[4].model: unexpected model name format
* BatchEmbedContentsRequest.requests[5].model: unexpected model name format
* BatchEmbedContentsRequest.requests[6].model: unexpected model name format
* BatchEmbedContentsRequest.requests[7].model: unexpected model name format
* BatchEmbedContentsRequest.requests[8].model: unexpected model name format


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chain.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'.")
    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [ ]:
57:55